In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('..'))

from fiat.train import cv_oof
from fiat.os import count
from fiat.arch.image_seg import unet
from tools.data import seg_train_gen, decode_img_seg, feature_dict

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:
csv_path = os.path.join('..', 'data', 'train.csv')
train_path = os.path.join('..', 'data', 'train_images')
tfr_path = os.path.join('..', 'tmp', 'cv', 'tfr')
ckpt_dir = os.path.join('..', 'tmp', 'cv', 'ckpt')

In [3]:
tfr_param = {'path': tfr_path,
             'count': count(train_path), 
             'feature_dict': feature_dict,
             'compression': 'GZIP',
             'c_level': 1}

gen = seg_train_gen(csv_path, train_path, sep='[_,]', nclass=4)

read_param = {'decode_raw': decode_img_seg, 
              'buffer_size': 100,
              'num_parallel_reads': 3}

arch = unet(num_layers=3, 
            feature_growth_rate=16,
            n_class=4,
            channels=3,
            padding='SAME',
            dropout_rate=0.25,
            active='sigmoid')

train_param = {'loss': 'dice', 'metric': 'dice',
               'optimizer': 'momentun', 'rate': 1e-6, 
               'epoch':1, 'batch_size': 1, 
               'early_stopping': 1,
               'reshape': [32, 200], 'reshape_method': 3, 
               'distrainable': False}

In [4]:
recorder = cv_oof(arch,
                  nflods=5,
                  read_param=read_param,
                  gen=gen,
                  train_param=train_param,
                  tfr_param=tfr_param,
                  ckpt_dir=ckpt_dir,
                  verbose=2, retrain=False, rewrite=False)

W1017 19:35:05.794657 139750419167040 deprecation_wrapper.py:119] From /home/leechh/code/Severstal/fiat/train.py:61: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W1017 19:35:05.836550 139750419167040 deprecation.py:323] From /home/leechh/anaconda3/envs/severstal/lib/python3.7/site-packages/tensorflow/python/data/util/random_seed.py:58: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1017 19:35:05.947486 139750419167040 deprecation.py:323] From /home/leechh/code/Severstal/fiat/train.py:74: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object

best epoch is 1,  train score is 0.9853570072070396, valid score is 0.9863538663351261


epoch 1, train loss: 0.9853, valid metric: 0.9854: 100%|██████████| 10054/10054 [03:35<00:00,  1.33s/it]


best epoch is 1,  train score is 0.9853208710759824, valid score is 0.9853999770721369


epoch 1, train loss: 0.9856, valid metric: 0.9852: 100%|██████████| 10054/10054 [03:37<00:00,  1.98s/it]


best epoch is 1,  train score is 0.9855660876851682, valid score is 0.9852410929489439


epoch 1, train loss: 0.9857, valid metric: 0.9848: 100%|██████████| 10054/10054 [03:39<00:00,  2.70s/it]


best epoch is 1,  train score is 0.9857234056140121, valid score is 0.9847725488737451


epoch 1, train loss: 0.9854, valid metric: 0.9855: 100%|██████████| 10056/10056 [03:50<00:00,  1.36s/it]

best epoch is 1,  train score is 0.9853959063404405, valid score is 0.9854624904929452
Finally, mean score is 0.9854459951445793, std is 0.0005142245905459239


In [ ]:
recorder = {'train_info': {'flod_0': {'train': [4294967296, 0.9844014687485503],
   'valid': [4294967296, 0.9854998715642718]},
  'flod_1': {'train': [4294967296, 0.9852299896615956],
   'valid': [4294967296, 0.9853130825569256]},
  'flod_2': {'train': [4294967296, 0.985502572254796],
   'valid': [4294967296, 0.9851784395763774]},
  'flod_3': {'train': [4294967296, 0.9853713586822712],
   'valid': [4294967296, 0.9843840416262998]},
  'flod_4': {'train': [4294967296, 0.9852895707947068],
   'valid': [4294967296, 0.9853626834643874]}},
 'besttrain': [0.9844014687485503,
  0.9852299896615956,
  0.985502572254796,
  0.9853713586822712,
  0.9852895707947068],
 'bestvalid': [0.9854998715642718,
  0.9853130825569256,
  0.9851784395763774,
  0.9843840416262998,
  0.9853626834643874],
 'mean': 0.9851476237576524,
 'std': 0.9851476237576524}

In [6]:
recorder = cv_oof(arch,
                  nflods=5,
                  read_param=read_param,
                  gen=gen,
                  train_param=train_param,
                  tfr_param=tfr_param,
                  ckpt_dir=ckpt_dir,
                  verbose=2, retrain=recorder['info'], rewrite=False)

W1017 19:55:00.146900 139750419167040 deprecation.py:323] From /home/leechh/anaconda3/envs/severstal/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
epoch 2, train loss: 0.9852, valid metric: 0.9862: 100%|██████████| 10054/10054 [03:49<00:00,  1.72s/it]


best epoch is 2,  train score is 0.9852367830319174, valid score is 0.9862481043655529


epoch 2, train loss: 0.9853, valid metric: 0.9854: 100%|██████████| 10054/10054 [03:40<00:00,  1.14s/it]


best epoch is 2,  train score is 0.98529500434383, valid score is 0.9853735650543571


epoch 2, train loss: 0.9855, valid metric: 0.9852: 100%|██████████| 10054/10054 [03:40<00:00,  8.02s/it]


best epoch is 2,  train score is 0.9855336628117665, valid score is 0.9852084135762642


epoch 2, train loss: 0.9857, valid metric: 0.9847: 100%|██████████| 10054/10054 [03:38<00:00,  1.60s/it]


best epoch is 2,  train score is 0.9856950978777957, valid score is 0.9847428369303971


epoch 2, train loss: 0.9854, valid metric: 0.9854: 100%|██████████| 10056/10056 [03:39<00:00,  1.61s/it]

best epoch is 2,  train score is 0.9853662292079596, valid score is 0.9854324647005955
Finally, mean score is 0.9854010769254333, std is 0.00048776974833902506
